In [1]:
import numpy as np

In [2]:
d = {
    "key": "value",
    "key1": "value1",
}

In [3]:
from box import ConfigBox

In [4]:
d2 = ConfigBox(d)

In [6]:
d2.key1

'value1'

In [7]:
def get_product(x:int, y:int)->int:
    return x*y

In [8]:
get_product(x = 2, y = 4)

8

In [9]:
get_product(x = 2, y = "4")

'44'

In [10]:
from ensure import ensure_annotations

In [11]:
@ensure_annotations
def get_product(x:int, y:int)->int:
    return x*y

In [12]:
get_product(x = 2, y = "4")

EnsureError: Argument y of type <class 'str'> to <function get_product at 0x0000026A1ECA28B8> does not match annotation type <class 'int'>

In [13]:
get_product(x = 2, y = 4)

8